In [1]:
import pandas as pd
import numpy as np
target='22-05-2024/'

In [4]:
df_rust = pd.read_csv(f'../data/{target}rust-gpu-times.csv')
df_rust['version'] = 'rust'
df_python = pd.read_csv(f'../data/{target}python-gpu-times.csv')
df_python['version'] = 'python'
# df_jax = pd.read_csv(f'../data/{target}jax-cpu-times.csv')
# df_jax['version'] = 'jax'
df = pd.concat([
  df_rust,
  df_python,
  # df_jax
])
df.columns

Index(['elapsed_real_time_seconds', 'kernel_cpu_seconds', 'user_cpu_seconds',
       'cpu_percentage', 'max_resident_set_size_kb', 'system_page_size_bytes',
       'major_page_faults', 'minor_page_faults',
       'involuntary_context_switches', 'voluntary_context_switches',
       'filesystem_input_reads', 'filesystem_output_writes',
       'command_name_args', 'version'],
      dtype='object')

# Time and CPU

In [5]:
df.groupby('version')['elapsed_real_time_seconds'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,106.7376,43.726547,18.49,70.525,100.685,146.6000,181.13
rust,100.0,72.4912,34.433319,11.42,44.190,66.365,106.0025,129.46


In [6]:
def trim_df(df, low=10, high=90):
    lower = np.percentile(df['elapsed_real_time_seconds'], low)
    upper = np.percentile(df['elapsed_real_time_seconds'], high)
    trimmed_df = df[df.elapsed_real_time_seconds.between(lower, upper)]
    return trimmed_df

In [8]:
df_trimmed = pd.concat([
  trim_df(df_rust),
  trim_df(df_python),
  # trim_df(df_jax)
])
df_trimmed.groupby('version')['elapsed_real_time_seconds'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,80.0,106.4145,34.762944,53.97,76.1275,100.685,140.3725,169.60
rust,80.0,72.1935,28.421489,32.23,47.4750,66.365,97.2650,121.23


In [9]:
df.groupby('version')['user_cpu_seconds'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,96.9475,43.284787,9.44,61.1000,91.235,136.355,170.35
rust,100.0,70.3365,33.924187,10.07,42.5475,64.345,103.725,126.97


In [10]:
df.groupby('version')['kernel_cpu_seconds'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,8.3171,0.307641,7.64,8.0900,8.275,8.5625,8.94
rust,100.0,1.8451,0.283665,1.38,1.6075,1.790,2.1225,2.35


In [11]:
df['cpu_percentage'] = df['cpu_percentage'].str[:-1].astype(int)
df.groupby('version')['cpu_percentage'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,97.82,0.925290,92.0,98.0,98.0,98.0,99.0
rust,100.0,98.89,1.127212,92.0,99.0,99.0,99.0,100.0


# Memory

'max_resident_set_size_kb',
'system_page_size_bytes',
'major_page_faults',
'minor_page_faults',

In [12]:
df.groupby('version')['max_resident_set_size_kb'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,2038480.84,24735.945550,1999752.0,2020149.0,2029888.0,2052719.0,2101872.0
rust,100.0,1834248.12,21501.340613,1803020.0,1820371.0,1828692.0,1841783.0,1895716.0


In [13]:
df.groupby('version')['major_page_faults'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
rust,100.0,0.04,0.4,0.0,0.0,0.0,0.0,4.0


In [14]:
df.groupby('version')['minor_page_faults'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,276988.52,14681.758467,245947.0,265027.75,272766.5,289033.50,308464.0
rust,100.0,225891.88,6715.316251,209292.0,221220.75,226509.0,230911.25,237841.0


# SO

'involuntary_context_switches',
'voluntary_context_switches', 
'filesystem_input_reads',
'filesystem_output_writes',

In [15]:
df.groupby('version')['involuntary_context_switches'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,158.60,39.309957,112.0,134.0,148.0,166.0,385.0
rust,100.0,124.09,23.274426,94.0,106.0,120.0,132.0,219.0


In [16]:
df.groupby('version')['voluntary_context_switches'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,62283.87,27215.739378,8745.0,40746.0,57547.5,86664.25,106397.0
rust,100.0,56669.97,28829.165723,6475.0,32807.0,52022.5,83911.25,99659.0


In [17]:
df.groupby('version')['filesystem_input_reads'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
rust,100.0,15.04,150.4,0.0,0.0,0.0,0.0,1504.0


In [18]:
df.groupby('version')['filesystem_output_writes'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,8.0,0.0,8.0,8.0,8.0,8.0,8.0
rust,100.0,8.0,0.0,8.0,8.0,8.0,8.0,8.0


# Rewards

In [21]:
df_rust = pd.read_csv(f'../data/{target}rust-gpu-rewards.csv')
df_python = pd.read_csv(f'../data/{target}python-gpu-rewards.csv')
# df_jax = pd.read_csv(f'../data/{target}jax-cpu-rewards.csv')
df = pd.concat([
  df_rust,
  df_python,
  # df_jax
])
df.columns

Index(['version', 'seed', 'training_steps', 'reward_mean', 'reward_std'], dtype='object')

In [22]:
df.groupby('version')['training_steps'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,28753.10,13344.304020,2000.0,18000.0,26000.0,41250.0,50000.0
rust,100.0,28777.36,14223.064721,4000.0,17000.0,26500.0,42250.0,50000.0


In [23]:
df.groupby('version')['reward_mean'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,444.323,136.869148,15.3,491.675,500.0,500.0,500.0
rust,100.0,454.857,117.488580,57.0,498.450,500.0,500.0,500.0


In [24]:
df.groupby('version')['reward_std'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,9.773689,24.212183,0.0,0.0,0.0,3.645482,136.460287
rust,100.0,9.097672,26.889829,0.0,0.0,0.0,2.187330,200.281830
